In [1]:
import pandas as pd
import os
import dotenv

dotenv.load_dotenv()
BASE_DIR = os.getcwd()

# Import Excel

In [2]:
excel_path = "disposition_code_ref_table.xlsx"

df = pd.read_excel(os.path.join(BASE_DIR, excel_path))
df.drop(columns=["Konkatenasi_Layer123"], inplace=True)

# 0. CLEAN DATA

In [3]:
def strip_all():
    df["Layer 1"] = df["Layer 1"].str.strip()
    df["Layer 2"] = df["Layer 2"].str.strip()
    df["Layer 3"] = df["Layer 3"].str.strip()

    df.fillna(" ", inplace=True)

df.loc[df["Layer 1"].str.contains("General Customer"), "Layer 1"] = df["Layer 1"].str.split(regex=r" (General Customer)").str[0]
strip_all()
# print(df["Layer 1"].value_counts())
df.replace({"Credit Cards": "Credit Card",
            "Kartu Kredit": "Credit Card",
            "Credit Card Suppl": "Credit Card Supplementary",
            "Kartu Debit": "Debit Card",
            "Watsapp": "Whatsapp",
            "Octo": "OCTO",
            "Transaksi  1500800": "Transaksi 1500800",
            "Pembesihan": "Pembersihan",
            r" /": "/",
            r"/ ": "/",
            r"/": "/ ",
            "Sevices": "Services",
            "Top UP": "Top Up",
            "Others": "Other",
            "E-mandate": "E-Mandate",
            "Belum Terima": "Tidak Terima",
            "Belum Menerima": "Tidak Terima",
            "Tidak Menerima": "Tidak Terima",
            "Vending Other": "Vending Machine Other",
            "CNY": "Customer Preferred Valas Chinese Yuan",
            "CP Blast": "Customer Preferred Blast",
            "CP Staff": "Customer Preferred Staff",
            "CPVALAS": "Customer Preferred Valas",
            "CPDORMANT": "Customer Preferred Dormant",
            "Done Contact Center Dok KTP Supplement": "Contact Center Dok KTP Supplement",
            "Done Telesales Dok KTP Supplement": "Telesales Dok KTP Supplement",
            # "KPR": "Kredit Pemilikan Rumah",
            # "KPM": "Kredit Pemilikan Mobil",
            "XKB ": "XKB: ",
            # "KTA": "Kredit Tanpa Agunan",
            # "PL": "Personal Loan",
            # "CDM": "CDM (Cash Deposit Machine)",
            # "CRM": "CRM (Cash Recycling Machine)",
            # "EDC": "EDC (Electronic Data Capture)",
            # "CNAF": "CNAF (CIMB Niaga Auto Finance)",
            # "KTA": "KTA (Kredit Tanpa Agunan)",
            # "ARO": "ARO (Automatic Roll Over)",
            # "PCL": "PCL (Permanent Credit Limit)",
            # "TCL": "TCL (Temporary Credit Limit)",
            # "POH": "POH (Pembantu Operasional Harian)",
            # "SPV": "SPV (Supervisor)",
            # "CC": "CC (Credit Card)",
            # "SLIK": "SLIK (Sistem Layanan Informasi Keuangan)",
            # "DSE": "DSE (Digital Sales Enablement)",
            },regex=True, inplace=True)
# print(df["Layer 1"].value_counts())

strip_all()
df.loc[df["Layer 3"].str.contains(": "), "Layer 3"] = df["Layer 3"].str.split(": ").str[1]
strip_all()
# print(df["Layer 1"].value_counts())
len(df["Layer 3"].unique())

482

## 1. Move from layer 3 to layer 2

In [4]:
def split_layer3to2(layer2_name:str, layer3_product_list:list, layer2_prefix:str="", layer2_postfix:str=""):
    for product in layer3_product_list:
        # Create boolean mask
        mask = (
                (df["Layer 2"] == layer2_name)
                & ((df["Layer 3"].str.startswith(product)) | (df["Layer 3"].str.endswith(product)))
        )

        # Update Layer 2 using boolean indexing
        df.loc[mask, "Layer 2"] = layer2_prefix + product + layer2_postfix

        # Create a second mask based on the updated Layer 2
        mask2 = ((
                        df["Layer 2"] == layer2_prefix + product + layer2_postfix)
                 & ((df["Layer 3"].str.startswith(product)) | (df["Layer 3"].str.endswith(product)))
             )

        # Update Layer 3 using vectorized string operations and boolean indexing
        df.loc[mask2, "Layer 3"] = df.loc[mask2, "Layer 3"].str.split(product).str[1]
    strip_all()

    return df

branchless_products = ["ATM/ CDM/ CRM",
                           "ATM",
                           "CDM",
                           "CRM",
                           "Kartu Debit",
                           "Debit Card",
                           "OCTO Clicks",
                           "OCTO Mobile",
                           "OCTO Pay",
                           "QRIS",
                           "Rekpon",
                           ]
split_layer3to2('Branchless', branchless_products)

,Layer 1,Layer 2,Layer 3
0,Complain,ATM,Barang Jatuh Di Area
1,Complain,ATM,Gagal Tarik
2,Complain,ATM,Gagal Transaksi
3,Complain,ATM,Jumlah Uang Tidak Sesuai
4,Complain,ATM,Kebersihan/ Kelengkapan
...,...,...,...
1163,Sales,KTA,Sales Done CC KTP Non Leads
1164,Sales,KTA,Sales Unidentified
1165,Eskalasi,Eskalasi,Kontak VDN Staff
1166,Eskalasi,Eskalasi,Kontak POH/ SPV


In [5]:
services_products = ["Biz Channel OCTO Merchant",
                     "Biz Channel",
                     "Social Media",
                     "Website",
                     "Contact Center",
                     "Whatsapp",
                     "Digital Lounge",
                     "Phone Banking",
                     "E-mail",
                     "EDC",
                     "Branch",
                     "Live Chat",
                     "Preferred",
                     "Chatbot",
                     "Transaksi 1500800",
                     "Vending Machine",
                     "Telesales",
                     "Restrukturisasi Kredit",
                     "Poin Xtra",
                     "Collection",
                     "Beasiswa",
                     "Asuransi",
                     ]
split_layer3to2('Services', services_products)

,Layer 1,Layer 2,Layer 3
0,Complain,ATM,Barang Jatuh Di Area
1,Complain,ATM,Gagal Tarik
2,Complain,ATM,Gagal Transaksi
3,Complain,ATM,Jumlah Uang Tidak Sesuai
4,Complain,ATM,Kebersihan/ Kelengkapan
...,...,...,...
1163,Sales,KTA,Sales Done CC KTP Non Leads
1164,Sales,KTA,Sales Unidentified
1165,Eskalasi,Eskalasi,Kontak VDN Staff
1166,Eskalasi,Eskalasi,Kontak POH/ SPV


In [6]:
lending_products = ["KTA/ PL",
                    "KPR",
                    "KPM",
                    "Refinancing (Multi Guna)",
                    "Xtra Kasbon",
                    ]
split_layer3to2('Lending', lending_products)
# , "Lending: ")

,Layer 1,Layer 2,Layer 3
0,Complain,ATM,Barang Jatuh Di Area
1,Complain,ATM,Gagal Tarik
2,Complain,ATM,Gagal Transaksi
3,Complain,ATM,Jumlah Uang Tidak Sesuai
4,Complain,ATM,Kebersihan/ Kelengkapan
...,...,...,...
1163,Sales,KTA,Sales Done CC KTP Non Leads
1164,Sales,KTA,Sales Unidentified
1165,Eskalasi,Eskalasi,Kontak VDN Staff
1166,Eskalasi,Eskalasi,Kontak POH/ SPV


In [7]:
funding_products = ["Tabungan",
                    "Giro",
                    "Deposito",
                    "Obligasi",
                    "Reksa Dana",
                    ]
split_layer3to2('Funding', funding_products)
# , "Funding: ")

,Layer 1,Layer 2,Layer 3
0,Complain,ATM,Barang Jatuh Di Area
1,Complain,ATM,Gagal Tarik
2,Complain,ATM,Gagal Transaksi
3,Complain,ATM,Jumlah Uang Tidak Sesuai
4,Complain,ATM,Kebersihan/ Kelengkapan
...,...,...,...
1163,Sales,KTA,Sales Done CC KTP Non Leads
1164,Sales,KTA,Sales Unidentified
1165,Eskalasi,Eskalasi,Kontak VDN Staff
1166,Eskalasi,Eskalasi,Kontak POH/ SPV


In [8]:
preferred_products = ["Poin Xtra",
                      "Customer Preferred Valas",
                      "Personal Credit Line (PCL)",
                      "PVI (Program Green Lane)",
                      ]
split_layer3to2('Program Preferred Blast', preferred_products)
#, "Preferred: ")

,Layer 1,Layer 2,Layer 3
0,Complain,ATM,Barang Jatuh Di Area
1,Complain,ATM,Gagal Tarik
2,Complain,ATM,Gagal Transaksi
3,Complain,ATM,Jumlah Uang Tidak Sesuai
4,Complain,ATM,Kebersihan/ Kelengkapan
...,...,...,...
1163,Sales,KTA,Sales Done CC KTP Non Leads
1164,Sales,KTA,Sales Unidentified
1165,Eskalasi,Eskalasi,Kontak VDN Staff
1166,Eskalasi,Eskalasi,Kontak POH/ SPV


In [9]:
other_products = ["Poin Xtra",
                  "Asuransi",
                  "Vending Machine",
                  "Collection",
                  "Beasiswa",
                  "Telesales",
                  "Restrukturisasi Kredit",
                  ]
split_layer3to2('Other', other_products)

,Layer 1,Layer 2,Layer 3
0,Complain,ATM,Barang Jatuh Di Area
1,Complain,ATM,Gagal Tarik
2,Complain,ATM,Gagal Transaksi
3,Complain,ATM,Jumlah Uang Tidak Sesuai
4,Complain,ATM,Kebersihan/ Kelengkapan
...,...,...,...
1163,Sales,KTA,Sales Done CC KTP Non Leads
1164,Sales,KTA,Sales Unidentified
1165,Eskalasi,Eskalasi,Kontak VDN Staff
1166,Eskalasi,Eskalasi,Kontak POH/ SPV


# 2. Remove postfix and prefix

In [10]:
def remove_prefix(layer, suffix):
    mask = df[layer].str.startswith(suffix)
    df.loc[mask, layer] = df.loc[mask, layer].str.replace(suffix, '').str.strip()

def remove_postfix(layer, suffix):
    df.loc[df[layer].str.endswith(suffix), layer] = df[layer].str.split(suffix).str[0].str.strip()

In [11]:
# split_layer3to2('Telesales Dok KTP Supplement', ["Telesales"])
# split_layer3to2('Done Telesales Dok KTP Supplement', ["Telesales Done"])
# split_layer3to2('Done Contact Center Dok KTP Supplement', ["Done Contact Center"])
# split_layer3to2('Contact Center Dok KTP Supplement', ["Contact Center"])
remove_postfix("Layer 3", "Contact Center")
remove_postfix("Layer 3", "Telesales")
remove_prefix("Layer 3", "Credit Card")

# 3. Move from layer 2 to layer 3

In [12]:
import numpy as np

def split_layer2to3(layer2_product_list:list, layer2_name:str, layer3_prefix:str="", layer3_postfix:str=""):
    for product in layer2_product_list:
        df["Layer 3"] = np.where(
                (df["Layer 2"] == product),
            layer3_prefix + product + layer3_postfix + df["Layer 3"],
            df["Layer 3"])
        df["Layer 2"] = np.where(
                (df["Layer 2"] == product),
            layer2_name,
            df["Layer 2"])
    strip_all()

In [13]:
credit_card_products = [
        "Credit Card",
        "Credit Card Aktivasi Accor Platinum",
        "Credit Card Aktivasi Accor World",
        "Credit Card Basic",
        "Credit Card General",
        "Credit Card Inactive & Dormant",
        "Credit Card Stickiness CPROT",
        "Credit Card Stickiness Non-Activated",
        "Credit Card Supplementary",
        "Credit Card Tendency Card",
    ]
split_layer2to3(credit_card_products, "Credit Card", "[", "] ")

remove_prefix("Layer 3", "[Credit Card]")

# 4: Feedback/Report and Report Status Checking -> Report Status

In [14]:
df.loc[df["Layer 1"] == "Report Status Checking", "Layer 3"] = "Status Open"
df.loc[df["Layer 1"] == "Feedback/ Report", "Layer 3"] = "Status Closed"
df.loc[df["Layer 1"] == "Report Status Checking", "Layer 1"] = "Report Status"
df.loc[df["Layer 1"] == "Feedback/ Report", "Layer 1"] = "Report Status"

# 5. Group Labels

In [15]:
def group_label(layer:str, list_of_labels:list[str], result_label:str):
    strip_all()
    for label in list_of_labels:
        df.loc[df[layer].str.contains(label), layer] = result_label
    strip_all()

group_label("Layer 1", ["Spam", "Ongoing", "Miscellaneous", "Test"], "Other")
group_label("Layer 1", ["Sales", "Campaign"], "Campaign/ Sales")

group_label("Layer 3", ["Promo/ Voucher", "Promo/ Voucher/milliage", "Promo/ Voucher/ Milliage", "Promo/ Voucher/Milliage"], "Promo/ Voucher/ Mileage")
group_label("Layer 3", ["Prank Call/ Spam Call/ Bogus Call"], "Spam")
group_label("Layer 3", ["Penerbitan", "Penggantian"], "Penerbitan/ Penggantian")

# 6. Swapping Layers

In [16]:
def swap_layers(layer_name_1: str, layer_name_2: str, layer_name_1_contain: str = "", layer_name_2_contain: str = ""):
    # Create a boolean mask for rows where both conditions are met
    mask = (df[layer_name_1].str.contains(layer_name_1_contain, na=False) &
            df[layer_name_2].str.contains(layer_name_2_contain, na=False))

    # Use .loc with simultaneous assignment for the swap
    df.loc[mask, [layer_name_1, layer_name_2]] = df.loc[mask, [layer_name_2, layer_name_1]].values

# test0 = df.loc[df["Layer 2"].str.contains("Whatsapp")]
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Salah Nama",layer_name_2_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Test",layer_name_2_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Chat lanjutan",layer_name_2_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Info belum lengkap",layer_name_2_contain="Other")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_2_contain="OCTO ")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Whatsapp")
swap_layers(layer_name_1="Layer 2",layer_name_2="Layer 3",layer_name_1_contain="Blast")

# test1 = df.loc[df["Layer 3"].str.contains("Whatsapp")]
# print(test0)
# print(test1)

# 7. Targeted Cleaning

In [17]:
df.loc[(df["Layer 1"] == "Other") & (df["Layer 2"] == "Salah layanan"), "Layer 2"] = "Other"
df.loc[(df["Layer 1"] == "Other") & (df["Layer 2"] == "Spam"), "Layer 2"] = "Other"
df.loc[df["Layer 3"] == "No", "Layer 3"] = "Tidak Setuju"
df.loc[df["Layer 2"] == "Contact", "Layer 2"] = "Contact Center"

# test7 = df.loc[(df["Layer 2"] == "Other") & (df["Layer 1"] == "Other")]
# test7

# 8. Cleaning Empty Data

In [18]:
strip_all()
df.loc[df["Layer 1"] == "", "Layer 1"] = None
df.loc[df["Layer 2"] == "", "Layer 2"] = None
df.loc[df["Layer 3"] == "", "Layer 3"] = None
df.fillna("Other", inplace=True)
df.drop_duplicates(inplace=True)
# df.sort_values(by=["Layer 1","Layer 2","Layer 3"],inplace=True)
# print(df.size//3)

## Get data By Product Name

In [19]:
df["Layer 1"].value_counts()

Layer 1
Complain           339
Inquiry            295
Request            290
Campaign/ Sales    131
Report Status       53
Compliment          31
Other                6
Eskalasi             3
Name: count, dtype: int64

In [20]:
get_data_by_product = df.loc[
    (df["Layer 2"] == "Credit Card")
    & (df["Layer 1"].str.contains("Complain"))
]

distinct_values = len(df["Layer 3"].unique())

## Create JSON from product to action

In [21]:
import json

product_to_action = {}

for _, row in df.iterrows():
    action, product, detail = row.iloc[0].strip(), row.iloc[1].strip(), row.iloc[2].strip()

    if product not in product_to_action:
        product_to_action[product] = []

    if action not in product_to_action[product]:
        product_to_action[product].append(action)

product_to_action_json = json.dumps(product_to_action, indent=2)
print(product_to_action_json)

{
  "ATM": [
    "Complain",
    "Inquiry",
    "Request"
  ],
  "CDM": [
    "Complain",
    "Inquiry",
    "Request"
  ],
  "CRM": [
    "Complain",
    "Inquiry",
    "Request"
  ],
  "Debit Card": [
    "Complain",
    "Compliment",
    "Report Status",
    "Inquiry",
    "Request"
  ],
  "OCTO Clicks": [
    "Complain",
    "Compliment",
    "Report Status",
    "Inquiry",
    "Request"
  ],
  "OCTO Mobile": [
    "Complain",
    "Compliment",
    "Report Status",
    "Inquiry",
    "Request"
  ],
  "OCTO Pay": [
    "Complain",
    "Inquiry",
    "Request"
  ],
  "QRIS": [
    "Complain",
    "Compliment",
    "Report Status",
    "Inquiry",
    "Request"
  ],
  "Credit Card": [
    "Complain",
    "Compliment",
    "Report Status",
    "Inquiry",
    "Request",
    "Campaign/ Sales"
  ],
  "Deposito": [
    "Complain",
    "Compliment",
    "Report Status",
    "Inquiry",
    "Request"
  ],
  "Funding": [
    "Complain",
    "Inquiry",
    "Request"
  ],
  "Giro": [
    "Complai

## Create JSON from (product_action) to detail

In [23]:
product_action_to_detail = {}

for _, row in df.iterrows():
    product_action, detail =f"{row.iloc[1].strip()}_{row.iloc[0].strip()}", row.iloc[2].strip()

    if product_action not in product_action_to_detail:
        product_action_to_detail[product_action] = []

    if detail not in product_action_to_detail[product_action]:
        product_action_to_detail[product_action].append(detail)

result = json.dumps(product_action_to_detail, indent=2)
print(len(product_action_to_detail))
print(result)

358
{
  "ATM_Complain": [
    "Barang Jatuh Di Area",
    "Gagal Tarik",
    "Gagal Transaksi",
    "Jumlah Uang Tidak Sesuai",
    "Kebersihan/ Kelengkapan",
    "Mesin Lambat",
    "Other",
    "Penipuan",
    "Uang Palsu",
    "Uang Rusak"
  ],
  "CDM_Complain": [
    "Barang Jatuh Di Area",
    "Gagal Setor",
    "Gagal Transaksi",
    "Kebersihan/ Kelengkapan",
    "Other",
    "Penipuan"
  ],
  "CRM_Complain": [
    "Barang Jatuh Di Area",
    "Gagal Setor",
    "Gagal Tarik",
    "Gagal Transaksi",
    "Kebersihan/ Kelengkapan",
    "Mesin Lambat",
    "Other",
    "Penipuan",
    "Uang Palsu",
    "Uang Rusak"
  ],
  "Debit Card_Complain": [
    "Tidak Terima PIN",
    "Kendala Replace/ Renewal",
    "Kendala Transaksi",
    "Other",
    "Pengiriman",
    "Promo",
    "Terblokir",
    "SMS Blast",
    "WhatsApp Blast",
    "Email Blast"
  ],
  "OCTO Clicks_Complain": [
    "Blokir",
    "Kendala Login",
    "Kendala Transaksi",
    "Other",
    "Pemulihan Password",
    "Pemuli